In [0]:

# est_average_active_days * est_active_users as est_total_active_days,
# est_average_bytes_per_session * est_active_users * est_average_session_count_per_user as est_total_bytes,
# est_active_users * est_average_session_count_per_user / est_share_of_session_product_in_main_category AS est_total_session_count_of_main_category,
# est_active_users * est_average_session_count_per_user * est_average_bytes_per_session / est_share_of_bytes_product_in_main_category AS est_total_bytes_of_main_category,
# est_active_users * est_average_time_per_user_milliseconds / est_share_of_time_product_in_main_category AS est_total_time_milliseconds_of_main_category,
# est_average_bytes_per_session * est_active_users * est_average_session_count_per_user * est_percentage_of_bytes_wifi_in_total * 1024 * 1024 * 1024 * 1024 AS est_wifi_bytes

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
usage_basic_before_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/"
granularity_list = ['monthly']
def test_bytes():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            df_after = spark.sql("""select count(1) from after_trans
            where (est_total_bytes = 0 and est_average_bytes_per_session <> 0 and est_average_session_count_per_user <> 0)
            or (est_total_session_count_of_main_category = 0 and est_share_of_session_product_in_main_category <> 0) 
            or (est_total_bytes_of_main_category = 0 and est_share_of_bytes_product_in_main_category <> 0)
            or (est_total_time_milliseconds_of_main_category = 0 and est_share_of_time_product_in_main_category <> 0)
            or (est_wifi_bytes = 0 and est_percentage_of_bytes_wifi_in_total <> 0 and est_average_session_count_per_user <> 0 and est_average_bytes_per_session <> 0)""").collect()
            if df_after[0][0] != 0:
                print granularity, date, "count", df_after[0][0]
            else:
                print granularity, date, "PASS"
                
def test_bytes_tmp():
    # monthly 2019-06-30 count 9646
    # monthly 2019-05-31 count 9061
    filter_str_after = "date between '2019-01-31' and '2019-12-31' and granularity_code = 'monthly' and product_key = 20600000006618 and device_code = 'android-phone' and country_code = 'US'"
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
    # spark.sql("select count(1) from after_trans where est_total_bytes = 0 and est_average_bytes_per_session <> 0 and est_average_session_count_per_user <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_total_session_count_of_main_category = 0 and est_share_of_session_product_in_main_category <> 0").show(10, False)
    spark.sql("select date, est_total_bytes_of_main_category*1024*1024 as total_bytes_of_category, est_share_of_bytes_product_in_main_category/1024/1024 as share_of_bytes, est_average_bytes_per_session, est_average_session_count_per_user from after_trans order by date asc").show(100, False)
    # spark.sql("select count(1) from after_trans where est_total_time_milliseconds_of_main_category = 0 and est_share_of_time_product_in_main_category <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_wifi_bytes = 0 and est_percentage_of_bytes_wifi_in_total <> 0 and est_average_session_count_per_user <> 0 and est_average_bytes_per_session <> 0").show(10, False)
    
    
def test_other_metrics():
    for granularity in granularity_list:
        for date in  DATE_GRANULARITY_MAPPINGLIST[granularity]:
            filter_str_after = "date = '{date}' and granularity_code = '{granularity}'".format(date=date, granularity=granularity)
            spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
            s = """select count(1) from after_trans
            where (est_install_penetration = 0  and est_open_rate <> 0)
            or (est_install_base = 0 and est_open_rate <> 0)
            or (est_population = 0 and est_usage_penetration <> 0)
            or (est_total_time_milliseconds = 0  and est_average_session_count_per_user <> 0)"""
            df_after = spark.sql(s).collect()
            if df_after[0][0] != 0:
                print granularity, date, "count", df_after[0][0]
            else:
                print granularity, date, "PASS"

def test_other_metrics_tmp():
    filter_str_after = "date = '2019-08-31' and granularity_code = 'monthly'"
    filter_str = "date ='2019-08-31' and granularity = 'monthly'"
    spark.read.format("delta").load(usage_basic_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
    # spark.sql("select count(1) from after_trans where est_install_penetration = 0  and est_open_rate <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_install_base = 0 and est_open_rate <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_population = 0 and est_usage_penetration <> 0").show(10, False)
    # spark.sql("select count(1) from after_trans where est_total_time_milliseconds = 0  and est_average_session_count_per_user <> 0").show(10, False)
    spark.sql("select count(1) from before_trans where est_total_time = 0 and est_average_session_per_user <> 0").show(10, False)
    spark.sql("select app_id, date, granularity, country_code, est_total_time, est_average_session_per_user, est_average_time_per_user, est_average_session_duration from before_trans where est_total_time = 0 and est_average_session_per_user <> 0 order by app_id desc limit 10").show(10, False)
    # spark.sql("select count(1) from before_trans").show(1, False)

test_bytes_tmp()



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/granularity_code=monthly/

In [0]:

domain_x_domain_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.cross-domain.v4/fact/granularity=m/"
# raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/date={date}/platform=2/"
# raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/"

for date in DATE_GRANULARITY_MAPPINGLIST["monthly"]:
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/date={date}/".format(date=date)

    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    # spark.sql("select * from test_raw limit 1").show()
    df = spark.sql("select count (1) as count from test_raw")
    print df.collect(), df.collectAsList(), df.first()
    
# spark.sql("select distinct product_name from domain_view").show(100, False)

# spark.sql("select distinct domain_id from test where country_code='us' and device_code = 'ip' order by domain_id desc").show(1000, False)


In [0]:

unified_retention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.retention-days.v6/fact/granularity=monthly/"
raw_retention_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_RETENTION_METRICS/version=v1.0.0/granularity=MONTH/"
spark.read.format("parquet").load(raw_retention_path).createOrReplaceTempView("test")
spark.sql("select * from test limit 1").show()
# spark.sql("select date, count(*), sum(estimate) from test where product_type_name = 'domain_only' group by date order by date desc").show(100, False)


In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY, PG_AA_HOSTS, PG_AA_NAME,PG_AA_ACCESS_ID,PG_AA_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

PG_AA_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2013, 1, 1)
end_date = datetime(2014, 12, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


# DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path='usage';
--select date, count(*) from usage_basic_kpi_fact_v7 where date >= '2020-06-28' and granularity = 'daily' and country_code = 'MO' and device_code = 'android-phone' group by date order by date;
--SELECT COUNT(country_code) FROM (SELECT DISTINCT country_code FROM usage_basic_kpi_fact_v7 WHERE date = '2020-07-19' AND granularity = 'daily' AND device_code IN ('android-phone', 'android-tablet')) country_tbl;
--select date, granularity, device_code, est_installs, est_install_base, est_usage_penetration / est_install_penetration, est_open_rate from usage_basic_kpi_fact_v6 where date >= '2020-09-05' limit 10;
--select date, granularity, device_code, count(*) from usage_basic_kpi_fact_v6 where date >='2020-08-29' and device_code like 'android%' group by date, granularity, device_code order by granularity, date, device_code;
--select est_average_active_users, country_code, est_population from usage_basic_kpi_fact_v1 where date = '2020-09-01' and device_code like 'ios%' and granularity = 'daily' and country_code = 'CN' limit 10;
--select * from usage_basic_kpi_fact_v1 where app_id = 711923939 and date in ('2019-06-01', '2019-06-08', '2019-06-15') and device_code like 'ios%' and granularity = 'daily' and country_code = 'CN' limit 10;
\x
select  sum(est_average_active_users) as sum_AU, sum(est_population) as sum_POP from usage_basic_kpi_fact_v1 where app_id = 20600004251992 and date between '2019-02-24' and '2019-06-29' and device_code = 'android-phone' and granularity = 'weekly' and country_code in ('CA', 'SG', 'ZA') group by app_id;
EOF

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType, ShortType, StringType, DateType


domain_basic = """select distinct domain_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct domain_id
    from mw.domain_basic_m
    where 
        date ='2020-09-30'
        and est_average_active_users <> 0
        and est_average_active_users is not null
$proxy$) tpl (domain_id bigint) order by domain_id desc;
"""

domain_retention = """
select distinct domain_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct domain_id
    from mw.domain_retention_m
    where 
       date ='2020-09-30'
        and retention_days = 0
        and estimate <> 0
        and estimate is not null
$proxy$) tpl (domain_id bigint) ;
"""

app_x_domain ="""
select distinct cross_domain_id as domain_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct cross_domain_id
    from mw.app_x_domain_m
    where 
        date ='2020-09-30'
        and est_cross_product_usage_penetration <> 0
        and est_cross_product_usage_penetration is not null
$proxy$) tpl (cross_domain_id bigint) ;
"""

domain_x_app ="""
select distinct domain_id as domain_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct domain_id
    from mw.domain_x_app_m
    where 
        date ='2020-09-30'
        and est_cross_product_usage_penetration <> 0
        and est_cross_product_usage_penetration is not null
$proxy$) tpl (domain_id bigint) ;
"""

domain_x_domain ="""
select distinct domain_id as domain_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct domain_id
    from mw.domain_x_domain_m
    where 
        date ='2020-09-30'
        and est_cross_product_usage_penetration <> 0
        and est_cross_product_usage_penetration is not null
$proxy$) tpl (domain_id bigint) ;
"""

domain_meta_sql = """
select domain_id, name, is_disabled, status, sensitive_status, first_supported_date from aa_domain_metadata where first_supported_date < '2020-10-01' and is_disabled = 'f' and sensitive_status = 0;
"""

def get_domain_meta():
    result = query(PG_AA_DSN, domain_meta_sql)
    df_data = [Row(domain_id=r[0][0], name=r[0][1], is_disabled=r[0][2], status=r[0][3], sensitive_status=r[0][4], first_supported_date=r[0][5]) for r in result]
    _schema =StructType([StructField("domain_id", LongType(), False),StructField("is_disabled", StringType(), True), 
    StructField("status", ShortType(), False),
    StructField("sensitive_status", ShortType(), False),
    StructField("first_supported_date", DateType(), False)])
    domain_meta = spark.createDataFrame(data=df_data, schema=_schema)
    return domain_meta
    
def get_plproxy_result(sql_str):
    result = query(PLPROXY_DSN, sql_str)
    df_data = [Row(domain_id=r[0]) for r in result]
    _schema =StructType([StructField("domain_id", LongType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    # df_plproxy.createOrReplaceTempView("plproxy_df_new")
    # spark.sql("select * from plproxy_df_new").show(10000, False)
    return df_plproxy
    
def get_dim():
    dim_path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.product.v2/dimension/"
    spark.read.format("delta").load(dim_path).createOrReplaceTempView("test_dim")

def get_raw_data_domain_basic():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BASIC_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    # spark.read.format("delta").load(dim_path).printSchema()
    
    # spark.read.format("delta").load(unified_source_path).createOrReplaceTempView("test_unified")
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct domain from test_raw  where metric_name = 'AU' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")
    
def get_raw_date_domain_retention():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_RETENTION_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct domain from test_raw  where metric_name = 'D0' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")
    
def get_raw_domain_unified_attribution():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_RETENTION_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct domain from test_raw  where metric_name = 'D0' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")
    
def get_raw_app_x_domain():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_APP_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct cross_domain as domain from test_raw  where metric_name = 'UP' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")
    
def get_raw_domain_x_app():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_APP_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct domain as domain from test_raw  where metric_name = 'UP' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")
    
def get_raw_domain_x_domain():
    raw_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_DOMAIN_METRICS/version=v1.0.0/granularity=MONTH/date=2020-09-30/"
    spark.read.format("parquet").load(raw_path).createOrReplaceTempView("test_raw")
    spark.sql("select distinct domain as domain from test_raw  where metric_name = 'UP' and value is not null and value <> 0 ").createOrReplaceTempView("raw_domain")
    spark.sql("select distinct dim.product_key from raw_domain raw left join test_dim dim on raw.domain = dim.product_name order by dim.product_key desc").createOrReplaceTempView("raw_domain_id")

def get_plproxy_data():
     df_plproxy=get_plproxy_result()
     df_plproxy.createOrReplaceTempView("plproxy_df_new")
     spark.sql("select count(distinct app_id) from plproxy_df_new").show(10, False)
    #  spark.sql("select * from plproxy_df_new order by rank asc").show(10000, False)
     
     spark.sql("select app_id, avg(estimate) as estimate from plproxy_df_new group by app_id order by estimate desc").show(10000, False)

def compare():
    get_dim()
    print "step 0"
    domain_meta = get_domain_meta()
    print "stepp 1"
    df_plproxy=get_plproxy_result(domain_x_domain)
    print "stepp 2"
    get_raw_domain_x_domain()
    df_plproxy.createOrReplaceTempView("plproxy_df_new")
    print "stepp 3"
    domain_meta.createOrReplaceTempView("domain_meta_df")
    print "step 4"
    spark.sql("select product_key from raw_domain_id raw join domain_meta_df meta on raw.product_key = meta.domain_id").createOrReplaceTempView("domain_after_filter")
    print "step 5"
    spark.sql("select distinct product_key as domain_id from domain_after_filter except select distinct domain_id from plproxy_df_new ").createOrReplaceTempView("domain_diff")
    print "step 6"
    # spark.sql("select distinct domain_id from plproxy_df_new except select distinct product_key as domain_id from raw_domain_id   ").createOrReplaceTempView("domain_diff")
    spark.sql("select count(1) from domain_diff").show(10, False)
    print "step 7"
    spark.sql("select * from domain_diff order by domain_id desc").show(100, False)
    
    
compare()
# unified_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/date=2020-05-16"
# spark.read.format("delta").load(unified_source_path).show(10)

In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

app_basic_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from mu.app_{granularity}
    where date='{date}'
        and kpi=1
        and estimate is not null
        and estimate > 0
$proxy$) tpl (count bigint);"""


segment_by_age_gender_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ag.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

segment_by_audience_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ad.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""
# check the country
# segment_by_product_sql = """
# select distinct store_id AS store_id from plproxy.execute_select_nestloop($proxy$ 
#     select distinct store_id AS store_id
#     from au.app_monthly
#     WHERE date='2015-01-31'
#         and kpi=1
#         and estimate is not null
#         and estimate <> 0

# $proxy$) tpl (store_id int) order by store_id;
# """

segment_by_product_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from au.app_{granularity}
    WHERE date='{date}'
    and kpi=1
    and estimate is not null
    and estimate <> 0

$proxy$) tpl (count bigint);
"""



app_cross_app = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ca.app_{granularity}
    where 
        date ='{date}'
        and kpi = 9
        and estimate is not null
        and estimate <> 0
$proxy$) tpl (count bigint);
"""

# app_retention = """
# select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
#     select count(1) as count
#     from rt.app_{granularity}
#     where 
#         date ='{date}'
# $proxy$) tpl (count bigint) ;
# """

app_retention = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from rt.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

granularity_list = ["monthly"]

def get_plproxy_data(sql_str):
    for granularity in granularity_list:
        for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
            result = query(PLPROXY_DSN, sql_str.format(granularity=granularity,date=date_str))
            count = result[0][0]
            print str(granularity)+str(" ")+str(date_str)+str(" ")+str(count)

# def get_plproxy_data(sql_str):
#     result = query(PLPROXY_DSN, sql_str)
#     count = result[0][0]
#     for r in result:
#         print(r[0])
#     # print str("monthly")+str(" ")+str("2017-05-31")+str(" ")+str(result)
            
def get_unified_data():
    print 1

get_plproxy_data(app_cross_app)  

In [0]:

basic_dump="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/granularity=monthly/"
basic="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/"
segment_by_age_gender_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity={granularity}/month={date}/"
segment_by_audience="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v2/fact/granularity={granularity}/month={date}/"
segment_by_product_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity={granularity}/month={date}"
app_cross_app="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity={granularity}/month={date}/"
app_retention="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
start_str="2013-01"
end_str="2014-12"

granularity_list = ["weekly"]

def get_dump_count(path_str):
    for granularity in granularity_list:
        for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
            spark.read.parquet(path_str.format(granularity=granularity, date=date_str[:-3])).filter("date = '{date}'".format(date=date_str)).createOrReplaceTempView("test")
            df = spark.sql("select count(1) from test where kpi=1 and estimate is not null and estimate <> 0").collect()
            
            print str(granularity)+str(" ")+str(date_str)+str(" ")+str(df[0][0])

def tmp_check():
    tmp_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=monthly/month=2014-12/"
    spark.read.parquet(tmp_path).createOrReplaceTempView("test")
    spark.sql("select distinct store_id from test order by store_id desc limit 100").show(100, False)
get_dump_count(segment_by_product_sql)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=monthly/month=2014-12/

In [0]:

basic_dump="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/granularity=monthly/"
basic="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/"
segment_by_age_gender_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=/"
segment_by_audience="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v2/fact/granularity=monthly/"
segment_by_product="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=weekly/"
app_cross_app="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity=monthly/"
app_retention="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
start_str="2015-01"
end_str="2017-05"

spark.read.parquet(app_retention).filter("date between '2020-08-01' and '2020-08-31' and app_id = 20600000009039 and store_id = 10 and gender=0 and age=0").createOrReplaceTempView("test")
spark.sql("select * from test limit 1").show()


In [0]:

seg_by_prdouct = "s3://aardvark-prod-pdx-mdm-to-int/crossapps_appused/version=1.0.0/"
seg_by_age_gender = "s3://aardvark-prod-pdx-mdm-to-int/audience_demographics/version=1.0.0/range_type=MONTH/"
spark.read.parquet(seg_by_prdouct).filter("date between '2020-09-01' and '2020-09-30' and range_type = 'MONTH'").createOrReplaceTempView("test")
spark.sql("select * from test where CROSSAPP_UP is not null and CROSSAPP_UP <> 0 limit 1").show()
spark.sql("select date, count(1) from test where CROSSAPP_UP is not null and CROSSAPP_UP <> 0 group by date order by date desc").show(10, False)

In [0]:

for date_str in DATE_GRANULARITY_MAPPINGLIST["monthly"]:
    try:
        # app x app
        result = query(PLPROXY_DSN, sql_template_a_x_a.format(date=date_str))
        count =result[0][1]
        affinity_sum = result[0][2]
        up_sum = result[1][2]
        # app x domain, domain x domain, domain x app
        for db_name in ["app_x_domain", "domain_x_domain", "domain_x_app"]:
            result = query(PLPROXY_DSN, sql_template.format(date=date_str, db_name=db_name))
            count += result[0][0]
            affinity_sum += result[0][1]
            up_sum += result[0][2]
        print "{},{},{:.20g},{:.20g}".format(date_str,count, up_sum, affinity_sum )
    except Exception, e:
        print "{},{}".format(date_str, "ERROR")


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
-- adhoc

select distinct domain_id from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct domain_id
    from mw.domain_basic_w
    where 
        date = '2020-05-16' and  est_average_active_users <> 0 and est_average_active_users is not null
\$proxy\$) tpl (domain_id bigint) order by domain_id asc;


EOF
#estimate DOUBLE PRECISION

In [0]:
%%sh
select country_code, sum(count_value) as count from plproxy.execute_select_nestloop(\$proxy\$ 
    select country_code, count(*) as count_value
    from mw.domain_basic_m 
    where 
        date ='2020-01-31' and  est_average_active_users <> 0 and est_average_active_users is not null group by country_code
\$proxy\$) tpl (country_code char(2), count_value bigint) group by country_code order by count desc;


select sum(count_value) from plproxy.execute_select_nestloop(\$proxy\$ 
    select count(*) as count_value 
    from mw.domain_basic_m 
    where 
        date ='2020-01-31' and  est_average_active_users <> 0 and est_average_active_users is not null and domain_id = 70010000386932
\$proxy\$) tpl ( count_value bigint);

In [0]:
%%sh
# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
# internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
           select  date, count(*) from plproxy.execute_select(\$proxy\$
SELECT * FROM mw.domain_x_domain_m where date = '2020-09-30' \$proxy\$)
 t (granularity char(1), device_code char(2), country_code char(2), date date, domain_id bigint, cross_domain_id bigint,
 est_cross_product_affinity real, est_cross_product_usage_penetration real) group by date order by date desc;


EOF

In [0]:
%%sh
# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
# internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
          select  date, count(*), cast(sum(est_cross_product_affinity) as double precision) from plproxy.execute_select(\$proxy\$
SELECT * FROM mw.domain_x_domain_m where date between '2020-01-01' and '2020-10-01' \$proxy\$)
 t (granularity char(1), device_code char(2), country_code char(2), date date, domain_id bigint, cross_domain_id bigint,
 est_cross_product_affinity real, est_cross_product_usage_penetration real) group by date order by date desc;


EOF

In [0]:
%%sh
# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
# internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
           select  count(*) from plproxy.execute_select(\$proxy\$
SELECT * FROM mu.app_daily where date between '2020-07-01' and '2020-07-31' and kpi = 1
and estimate is not null and estimate <> 0 \$proxy\$)
 t (device_id smallint, store_id int, date date, kpi smallint, app_id bigint, estimate double precision);







EOF

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity==monthly/

In [0]:
%%sh
